In [2]:
import pandas as pd
import json

ticker_file = open('../Data/Raw Request Data/GOOGL_Annual_Financials.json')

ticker_json = json.load(open('../Data/Raw Request Data/GOOGL_Annual_Financials.json'))

pre_df = ticker_json['results'][0]

Google_DF = pd.DataFrame(ticker_json['results'])

In [59]:
Google_DF.head(3)

,id,start_date,end_date,filing_date,timeframe,fiscal_period,fiscal_year,cik,sic,tickers,company_name,source_filing_url,source_filing_file_url,financials
0,0001652044:2016:FY,2016-01-01,2016-12-31,2017-02-03,annual,FY,2016,0001652044,7370,"[GOOG, GOOGL]",Alphabet Inc.,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'comprehensive_income': {'other_comprehensive...
1,0001652044:2017:FY,2017-01-01,2017-12-31,2018-02-06,annual,FY,2017,0001652044,7370,"[GOOG, GOOGL]",Alphabet Inc.,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'balance_sheet': {'current_liabilities': {'va...
2,0001652044:2018:FY,2018-01-01,2018-12-31,2019-02-05,annual,FY,2018,0001652044,7370,"[GOOG, GOOGL]",Alphabet Inc.,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'income_statement': {'benefits_costs_expenses...


In [56]:
Google_Stripped = Google_DF.drop(['source_filing_url', 'source_filing_file_url', 'cik', 'timeframe'], axis = 1)
Google_Stripped.set_index('id', inplace = True)
Google_Stripped

,start_date,end_date,filing_date,fiscal_period,fiscal_year,sic,tickers,company_name,financials
id,,,,,,,,,
0001652044:2016:FY,2016-01-01,2016-12-31,2017-02-03,FY,2016,7370,"[GOOG, GOOGL]",Alphabet Inc.,{'comprehensive_income': {'other_comprehensive...
0001652044:2017:FY,2017-01-01,2017-12-31,2018-02-06,FY,2017,7370,"[GOOG, GOOGL]",Alphabet Inc.,{'balance_sheet': {'current_liabilities': {'va...
0001652044:2018:FY,2018-01-01,2018-12-31,2019-02-05,FY,2018,7370,"[GOOG, GOOGL]",Alphabet Inc.,{'income_statement': {'benefits_costs_expenses...
0001652044:2019:FY,2019-01-01,2019-12-31,2020-02-04,FY,2019,7370,"[GOOG, GOOGL]",Alphabet Inc.,{'income_statement': {'participating_securitie...
0001652044:2020:FY,2020-01-01,2020-12-31,2021-02-03,FY,2020,7370,"[GOOG, GOOGL]",Alphabet Inc.,{'cash_flow_statement': {'net_cash_flow_from_o...
0001652044:2021:FY,2021-01-01,2021-12-31,2022-02-02,FY,2021,7370,"[GOOG, GOOGL]",Alphabet Inc.,{'comprehensive_income': {'comprehensive_incom...


In [211]:
comp_columns = ['other_comprehensive_income_loss_attributable_to_parent', 'comprehensive_income_loss_attributable_to_noncontrolling_interest', 'comprehensive_income_loss', 'other_comprehensive_income_loss', 'comprehensive_income_loss_attributable_to_parent']

comprehensive_income_df = pd.DataFrame(columns = comp_columns)

income_statement_df = pd.DataFrame()

cash_flow_df = pd.DataFrame()

balance_sheet_df = pd.DataFrame() 


for index, rows in Google_Stripped.iterrows():
    financial_statement_list = []
    for statement_name, statement_values in Google_Stripped['financials'][index].items():
        financial_dict = {}
        financial_data = {}
        financial_statement_list.append(str(statement_name))
        # print(f'###########{statement_name} for filing {index}##########')
        column_names = []
        for line_item, line_value in statement_values.items():
            column_names.append(line_item)
            for key, value in line_value.items():
                if key == 'value':
                    financial_data[line_item] = value
        statement_series = pd.Series(data = financial_data, name = index)
        if statement_name == 'comprehensive_income':
            comprehensive_income_df = pd.concat(objs = [comprehensive_income_df, statement_series.to_frame().T])
        
        elif statement_name == 'income_statement':
            income_statement_df = pd.concat(objs = [income_statement_df, statement_series.to_frame().T])
           
        elif statement_name == 'cash_flow_statement':
            cash_flow_df = pd.concat(objs = [cash_flow_df, statement_series.to_frame().T])

        elif statement_name == 'balance_sheet':
            balance_sheet_df = pd.concat(objs = [balance_sheet_df, statement_series.to_frame().T])

        else:
            print('Error')
            continue
print(income_statement_df)

                    cost_of_revenue  net_income_loss_attributable_to_parent  \
0001652044:2016:FY     3.513800e+10                            1.947800e+10   
0001652044:2017:FY     4.558300e+10                            1.266200e+10   
0001652044:2018:FY     5.954900e+10                            3.073600e+10   
0001652044:2019:FY     7.189600e+10                            3.434300e+10   
0001652044:2020:FY     8.473200e+10                            4.026900e+10   
0001652044:2021:FY     1.109390e+11                            7.603300e+10   

                    gross_profit  \
0001652044:2016:FY  5.513400e+10   
0001652044:2017:FY  6.527200e+10   
0001652044:2018:FY  7.727000e+10   
0001652044:2019:FY  8.996100e+10   
0001652044:2020:FY  9.779500e+10   
0001652044:2021:FY  1.466980e+11   

                    net_income_loss_available_to_common_stockholders_basic  \
0001652044:2016:FY                                       1.947800e+10        
0001652044:2017:FY                   

In [25]:
Google_Stripped['financials'].explode('comprehensive_income')

0     comprehensive_income
1         income_statement
2      cash_flow_statement
3            balance_sheet
4            balance_sheet
5      cash_flow_statement
6         income_statement
7     comprehensive_income
8         income_statement
9      cash_flow_statement
10    comprehensive_income
11           balance_sheet
12        income_statement
13    comprehensive_income
14     cash_flow_statement
15           balance_sheet
16     cash_flow_statement
17        income_statement
18           balance_sheet
19    comprehensive_income
20    comprehensive_income
21           balance_sheet
22        income_statement
23     cash_flow_statement
Name: financials, dtype: object

In [20]:
for key, value in Google_Stripped['financials'][0]['comprehensive_income'].items():

    for key, value in Google_Stripped['financials'][0]['comprehensive_income'][key].items():
        print(f'-----------{key}-----------')
        for dollar_value in value:
            print(dollar_value)


-----------value-----------


TypeError: 'float' object is not iterable

In [39]:
ticker_json

{'results': [{'id': '0001652044:2016:FY',
   'start_date': '2016-01-01',
   'end_date': '2016-12-31',
   'filing_date': '2017-02-03',
   'timeframe': 'annual',
   'fiscal_period': 'FY',
   'fiscal_year': '2016',
   'cik': '0001652044',
   'sic': '7370',
   'tickers': ['GOOG', 'GOOGL'],
   'company_name': 'Alphabet Inc.',
   'source_filing_url': 'https://api.polygon.io/v1/reference/sec/filings/0001652044-17-000008',
   'source_filing_file_url': 'http://api.polygon.io/v1/reference/sec/filings/0001652044-17-000008/files/goog-20161231.xml',
   'financials': {'comprehensive_income': {'other_comprehensive_income_loss_attributable_to_parent': {'value': -528000000.0,
      'unit': 'USD',
      'label': 'Other Comprehensive Income/Loss Attributable To Parent',
      'order': 600},
     'comprehensive_income_loss_attributable_to_noncontrolling_interest': {'value': 0,
      'unit': 'USD',
      'label': 'Comprehensive Income/Loss Attributable To Noncontrolling Interest',
      'order': 200},
    